In [10]:
import pandas as pd
import pickle
import numpy as np
import datetime as dt
import pprint as pp
import matplotlib as plt
import pandas as pd
from IPython.display import display

from sklearn.preprocessing import StandardScaler


In [3]:
df = pd.read_pickle('../data/player_game_table.pkl')
df2 = pd.read_pickle('./player_vs_table_3.pkl')

features = ['ast','blk','min','pts','ftm','fga','fgm','reb','stl','to',
            'fantasy_points']
matchup_features = ['reb','oreb','dreb','ast','tov','stl','blk','pts',"fantasy_points"]

In [4]:
def create_input_features(pid, n, df, feats):
    rows = df.loc[ (df['player_id'] == pid) & (df['season'] == "2017-2018")]
    rows = rows.sort_values("game_date")

    X = np.zeros(( (len(rows) - (n), (len(feats)*n)+len(matchup_features))))
    y= np.zeros((len(rows) - (n)))

    for i in range(n, (len(rows) - 1)):
        last_n = rows.iloc[(i - n) : i]
        #X[(i - n),0] = pid
        fp_opp = []
        min_opp = []
        
        for a,feat in enumerate(feats):
            start_idx = a*n
            end_idx = start_idx + n
            X[(i - n),start_idx:end_idx] = last_n[feat].tolist()
        for b,mufeat in enumerate(matchup_features):
            #Calculate matchup statistics
            current_game = df.iloc[i].game_id
            pids = df.loc[df.game_id == current_game].player_id
            for pid in pids:
                x = df2.loc[(df2.player_id =='2544') & (df2.player_vs_id == pid) & (df2.season == '2016-17')]
                if len(x)>0:
                    fp_opp.append(x[mufeat].values[0])
                    min_opp.append(x["min"].values[0])
                    
            feature_average = np.ma.average(fp_opp,weights = min_opp)
            X[(i-n), len(feats)*n+b] = feature_average
            
        y[i-n] = rows.iloc[i]["fantasy_points"]
        
    return X, y

In [5]:
X, y = create_input_features('2544', 5, df, features)
lebron_dict = {}
lebron_dict["X"] = X
lebron_dict["y"] = y

with open('lebron_2018.pkl', 'wb') as fp:
    pickle.dump(lebron_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)




In [6]:
with open('lebron_2018.pkl', 'rb') as fp:
    lebron = pickle.load(fp)
    
scaler = StandardScaler()

X = lebron["X"]
y = lebron["y"]

X = scaler.fit_transform(X,y)
X = X[:-1]
y = y[:-1]
print(np.shape(X))

(76, 64)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import ParameterSampler

train_split = 3/6
train_n = int(len(y)*train_split)


param_grid={"max_depth": [None],
            "max_features": list(range(1,53)),
            "min_samples_split": list(range(2,50)),
            "n_estimators": list(range(1,128)),
            "bootstrap": [True, False],
            "random_state": [0],
            "criterion": ["mae", "mse"]}
         
rfr = RandomForestRegressor()

best_score = 0;
current_score = 0
for g in ParameterSampler(param_grid, n_iter=10000):
    rfr.set_params(**g)
    rfr.fit(X[:train_n],y[:train_n])
    rfr.predict(X[train_n:])
    current_score = rfr.score(X[train_n:],y[train_n:])
    # save if best
    if current_score > best_score:
        best_score = current_score
        best_grid = g

print (best_score) 
print (best_grid)